In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Read the image on which we want to apply face detection
image = cv2.imread('media/M4/john1.jpg')

# Initilizing the haar classifier with the face detector model
face_cascade = cv2.CascadeClassifier('media/M4/haarcascade_frontalface_default.xml')  


In [4]:
def transfer_2(image, replacement_face, returnimage=False, expand=45,thresh=180):

    # perform the detection , here we are using 1.3 scale factor and 5 min neighbours
    faces = face_cascade.detectMultiScale(image, 1.3, 5)    
    for (x,y,w,h) in faces:
        
        # Since our original face_replacement image is small we expand it by a custom value
        w= w+expand                                         
        h= h+expand    
        
        # Since we just increased our original w,h by add expand we must now center the image  
        x = int(x-(expand/2))                                 
        y = int(y-(expand/2))                                 
            
        replacement_face_copy = cv2.resize(replacement_face, (w,h))  
        rows,cols,channels =  replacement_face_copy.shape
        roi = image[y:y+h, x:x+w]
        img2gray = cv2.cvtColor(replacement_face_copy,cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(img2gray, thresh, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        
        # This is to handle faces going out of the boundry cause they will throw an error
        try:
            img_bg = cv2.bitwise_and(roi,roi,mask = mask)
        except:
            continue
        img_fg = cv2.bitwise_and(replacement_face_copy,replacement_face_copy,mask = mask_inv)
        combined = cv2.add(img_bg,img_fg)
        image[y:y+h, x:x+w] = combined
        

    if returnimage:
        return image
    else:
        cv2.imshow('img',image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [5]:
replacement_face = cv2.imread('media/M4/thejoker.jpg')
cap = cv2.VideoCapture(0)

while 1:
        ret, img = cap.read()
        img = transfer_2(img,  replacement_face, returnimage = True,expand=100,thresh=240)
        img = cv2.flip(img, 1)
        
        cv2.imshow('img',img)
        k = cv2.waitKey(1) 
        if k == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()